In [1]:
import numpy as np
import pandas as pd
import os

from leo_vetter.stellar import quadratic_ldc
from leo_vetter.main import TCELightCurve
from leo_vetter.thresholds import check_thresholds

import lightkurve as lk
from astroquery.mast import Catalogs

### Using Custom Vetting Thresholds

Pass-fail thresholds are meant to mimic human decision-making, but everyone has their own tolerance for what is considered a planet candidate. You may find that the current thresholds are too strong or weak.

You may also want to tune the pass-fail thresholds for your specific use-case. For example, if you are mostly interested in identifying new planets, you may want to use more lenient thresholds to recover more planets (at the cost of having a higher false positive/false alarm rate).

This example will briefly cover how you can tell the vetter to use a different set of thresholds.

In [2]:
tic = 231663901
per = 1.430363 
epo = 1338.885
dur = 0.069

# If you ran the previous notebooks, the metrics were saved to a file, so we can skip recomputing them here.
save_file = f"{tic}.1.metrics"

if os.path.exists(save_file):
    print("Loading saved metrics from file.")
    metrics = pd.read_csv(save_file, index_col=False)
    metrics = metrics.to_dict("records")[0]
else:
    # Download light curve
    search_result = lk.search_lightcurve(f"TIC {tic}", mission="TESS", author="TESS-SPOC")
    lcs = search_result.download_all()
    lc = lcs.stitch()
    lc = lc[~np.isnan(lc["flux"]) & (lc["quality"] == 0)]
    transit_mask = lc.create_transit_mask(transit_time=epo, period=per, duration=dur)
    lc_flat = lc.flatten(mask=transit_mask)

    time = np.asarray(lc_flat["time"].value)
    raw = np.asarray(lc["flux"].value)
    flux = np.asarray(lc_flat["flux"].value)
    flux_err = np.asarray(lc_flat["flux_err"].value)

    # Get stellar properties
    result = Catalogs.query_criteria(catalog="TIC", ID=tic)
    star = {}
    star["tic"] = tic
    for key in ["rad","mass","rho","Teff","logg"]:
        star[key] = float(result[key])
        star["e_"+key] = float(result["e_"+key])

    star["u1"], star["u2"] = quadratic_ldc(star["Teff"], star["logg"])

    # Load the TCELightCurve class
    tlc = TCELightCurve(tic, time, raw, flux, flux_err, per, epo, dur, planetno=1)
    
    # Flux-level vetting
    tlc.compute_flux_metrics(star, verbose=True)
    metrics = tlc.metrics

Estimating SES and MES time series...


/Users/batfan/miniconda3/envs/exoplanet/lib/python3.12/site-packages/leo_vetter/utils.py:17: RuntimeWarning: invalid value encountered in scalar divide
  mean = np.sum(w * y) / np.sum(w)
/Users/batfan/miniconda3/envs/exoplanet/lib/python3.12/site-packages/leo_vetter/utils.py:23: RuntimeWarning: divide by zero encountered in scalar divide
  err = 1 / np.sqrt(np.sum(w))


Fitting linear, trapezoid, and transit models...
Running SWEET test...
Getting odd-even metrics...
Checking individual transit events...
Running modshift...
Estimating derived parameters...
Done!


/Users/batfan/miniconda3/envs/exoplanet/lib/python3.12/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/batfan/miniconda3/envs/exoplanet/lib/python3.12/site-packages/leo_vetter/individual.py:106: RuntimeWarning: divide by zero encountered in scalar divide
  tlc.redchi2[i] = chi2 / (np.sum(fit_epoch) - 6)


Now that you've got your metrics, you have two options for applying custom thresholds:

a) For easy custom thresholds: simply define a dict containing the thresholds you want (the defaults are provided in the `thresholds.py` file and copied here) and use those when you check the metrics!

In [3]:
use_thresholds = {
    "MES": 6.2,
    "N_transit": 3,
    "SHP": 0.5,
    "MS1": 1,
    "MS2": 1,
    "MS3": 1,
    "chases": 0.78,
    "DMM": 1.5,
    "max_SES_to_MES": 0.8,
    "AIC1": -60,
    "AIC2": -30,
    "SWEET": 15,
    "ASYM": 8,
    "CHI": 7.8,
    "frac_gap": 0.5,
    "V_shape": 1.5,
    "size": 22,
    "MS4": 0,
    "MS5": -1,
    "MS6": -1,
    "offset": 15,
}

FA = check_thresholds(metrics, "FA", verbose=True, thresholds=use_thresholds)
FP = check_thresholds(metrics, "FP", verbose=True, thresholds=use_thresholds)

Passed all FA tests
Passed all FP tests


b) For more comprehensive custom thresholds: copy the `thresholds.py` file to your working directory, for example into a file called `user_thresholds.py`, and edit it directly. You can change or even add new tests, as long as they meet the following criteria:

1. Takes metrics and thresholds as an input
2. Outputs a mask (True if the test fails; False otherwise) and a message to be communicated if the test fails

Add the new test to the list of either FA or FP tests in the function `check_thresholds`. To use this file instead of the default file built in to LEO, run the following:

In [4]:
from user_thresholds import check_thresholds

FA = check_thresholds(metrics, "FA", verbose=True)
FP = check_thresholds(metrics, "FP", verbose=True)

Passed all FA tests
Passed all FP tests
